# Access Control List

## What is Access Control Lists?

Based on your business needs, you may have to permit or deny the access of users and devices in or out of your network. With Access Control List (ACL) you can restrict the traffic in a network.

ACL uses rules to determine whether to permit or deny traffic based on source and destination IP addresses, source port and destination port, and the protocol of the packet.

Use ACL to:
* Filter incoming or outgoing packets on an interface
* Filter packets for mirroring
* Redirect traffic as required
* Restrict the contents of routing updates
* Limit debug output based on an address or protocol. 
* Identify or classify traffic for advanced features, such as congestion avoidance, congestion management, and priority and custom queueing. 

## Configuration Steps


* [Bring Up Emulator](#step1)
* [Create and Apply ACL](#step2)
* [Verify the ACL configuration](#step3)
* [Modify the ACL configuration](#step4)



### <a name="step1"></a>Bring up Your Emulator


In the cell below, the python module acl contains the code to set up the python environment, the simulated 2-router topology. The sim.start() API brings up the emulator as per the network topology. Refer the acl.py file in this repository to have a look at the python module.

You have to execute the cell below by clicking the play button.

In [ ]:
from acl import *
sim = Vxr()
sim.no_image_copy=True
sim.clean()
print("Sim clean: Done")
print("Simulation starting. Please wait for the Sim status message. This may take 3-10 minutes.")

try:
    sim.start(cfg)
    ports=sim.ports()
    print(ports)
    status = sim.status()
    print("Sim status: ", status)
except Exception as err:
    print("Sim launch failed (%s)" % str(err))

>Play the cell below to obtain the telnet and ssh IP-addresses and ports to access the devices.

In [2]:
print('Consoles can be reached by:')
print(get_telnet_cmd(sim, 'r1'), '\n', get_telnet_cmd(sim, 'r2'))
print('or better:')
print(get_ssh_cmd(sim, 'r1'), '\n', get_ssh_cmd(sim, 'r2'))
print('The password is cisco123')

Consoles can be reached by:
telnet 172.17.0.2 41147 
 telnet 172.17.0.2 44917
or better:
ssh cisco@172.17.0.2 -p62147 
 ssh cisco@172.17.0.2 -p63048
The password is cisco123


>The cell below accesses the telnet consoles of the routers so that we can apply the required ACL configuration from this notebook.

In [3]:
import telnetlib
ports = sim.ports()
loginr1 = telnetlib.Telnet(str(ports['r1']['HostAgent']) , str(ports['r1']['serial0']))
loginr2 = telnetlib.Telnet(str(ports['r2']['HostAgent']) , str(ports['r2']['serial0']))



### Create and Apply ACL <a name="step2"></a> 

>In this notebook, let's see how to add rules using ACL to permit or deny traffic from a particular IP address. In this example, we will first allow traffic from 198.51.100.2.  Play the cell below to create the access list, then define the rules to permit trafffic from 198.51.100.2. Finally, apply the rules to an interface FourHundredGigE0/0/0/0.

In [4]:
loginr1.write(('''
configure
ipv4 access-list myacl
1 permit ipv4 host 198.51.100.2 any
exit
interface FourHundredGigE0/0/0/0
ipv4 access-group myacl ingress
commit
root
exit

''').encode('ascii'))
line = loginr1.read_until(b'/r/n',2)
print("***** VIEW FROM TELNET CONSOLE *****")
print(line.decode())

***** VIEW FROM TELNET CONSOLE *****

RP/0/RP0/CPU0:P1#configure
Fri Apr 23 05:56:17.264 UTC
RP/0/RP0/CPU0:P1(config)#ipv4 access-list myacl
RP/0/RP0/CPU0:P1(config-ipv4-acl)#1 permit ipv4 host 198.51.100.2 any
RP/0/RP0/CPU0:P1(config-ipv4-acl)#exit
RP/0/RP0/CPU0:P1(config)#interface FourHundredGigE0/0/0/0
RP/0/RP0/CPU0:P1(config-if)#ipv4 access-group myacl ingress
RP/0/RP0/CPU0:P1(config-if)#commit
Fri Apr 23 05:56:17.763 UTC
RP/0/RP0/CPU0:P1(config-if)#root
RP/0/RP0/CPU0:P1(config)#exit
RP/0/RP0/CPU0:P1#
RP/0/RP0/CPU0:P1#


### Verify ACL Configuration <a name="step3"></a> 

>Play the below cell to verify if the ACL configuration applied on the router.

In [5]:
loginr1.write(('''
show ip access-lists
''').encode('ascii'))
line = loginr1.read_until(b'/r/n',2)
print("***** VIEW FROM TELNET CONSOLE *****")
print(line.decode())

***** VIEW FROM TELNET CONSOLE *****

RP/0/RP0/CPU0:P1#show ip access-lists
Fri Apr 23 05:56:19.434 UTC
ipv4 access-list myacl
 1 permit ipv4 host 198.51.100.2 any
RP/0/RP0/CPU0:P1#


> Let's see if the traffic is allowed from 198.51.100.2 using the Ping command.

In [6]:
loginr1.write(('''
ping ipv4 198.51.100.2 count 20
''').encode('ascii'))
line = loginr1.read_until(b'/r/n',2)
print("***** VIEW FROM TELNET CONSOLE *****")
print(line.decode())

***** VIEW FROM TELNET CONSOLE *****

RP/0/RP0/CPU0:P1#ping ipv4 198.51.100.2 count 20
Fri Apr 23 05:56:21.438 UTC
Type escape sequence to abort.
Sending 20, 100-byte ICMP Echos to 198.51.100.2, timeout is 2 seconds:
!!!!!!!!!!!!!!!!!!!!
Success rate is 100 percent (20/20), round-trip min/avg/max = 10/12/34 ms
RP/0/RP0/CPU0:P1#


Yes, Ping is successful and the traffic is permitted.

### Modify the ACL Configuration <a name="step3"></a> 

Now, let's restrict the traffic from 198.51.100.2 by modifying the ACL configuration.

In [7]:
loginr1.write(('''
configure
ipv4 access-list myacl
1 deny ipv4 host 198.51.100.2 any
exit
interface FourHundredGigE0/0/0/0
ipv4 access-group myacl ingress
commit
root
exit

''').encode('ascii'))
line = loginr1.read_until(b'/r/n',2)
print("***** VIEW FROM TELNET CONSOLE *****")
print(line.decode())

***** VIEW FROM TELNET CONSOLE *****

RP/0/RP0/CPU0:P1#configure
Fri Apr 23 05:56:25.559 UTC
RP/0/RP0/CPU0:P1(config)#ipv4 access-list myacl
RP/0/RP0/CPU0:P1(config-ipv4-acl)#1 deny ipv4 host 198.51.100.2 any
RP/0/RP0/CPU0:P1(config-ipv4-acl)#exit
RP/0/RP0/CPU0:P1(config)#interface FourHundredGigE0/0/0/0
RP/0/RP0/CPU0:P1(config-if)#ipv4 access-group myacl ingress
RP/0/RP0/CPU0:P1(config-if)#commit
Fri Apr 23 05:56:26.079 UTC
RP/0/RP0/CPU0:P1(config-if)#root
RP/0/RP0/CPU0:P1(config)#exit
RP/0/RP0/CPU0:P1#
RP/0/RP0/CPU0:P1#


### Veriy the configuration with a show command.

In [8]:
loginr1.write(('''
show ip access-lists
''').encode('ascii'))
line = loginr1.read_until(b'/r/n',2)
print("***** VIEW FROM TELNET CONSOLE *****")
print(line.decode())

***** VIEW FROM TELNET CONSOLE *****

RP/0/RP0/CPU0:P1#show ip access-lists
Fri Apr 23 05:56:27.564 UTC
ipv4 access-list myacl
 1 deny ipv4 host 198.51.100.2 any
RP/0/RP0/CPU0:P1#


> Let's see if the traffic is allowed from 198.51.100.2 using the Ping command.

In [9]:
loginr1.write(('''
ping ipv4 198.51.100.2 count 5
''').encode('ascii'))
line = loginr1.read_until(b'/r/n',2)
print("***** VIEW FROM TELNET CONSOLE *****")
print(line.decode())

***** VIEW FROM TELNET CONSOLE *****

RP/0/RP0/CPU0:P1#ping ipv4 198.51.100.2 count 5
Fri Apr 23 05:56:32.121 UTC
Type escape sequence to abort.
Sending 5, 100-byte ICMP Echos to 198.51.100.2, timeout is 2 seconds:



> Router R1 is unable to connect to 198.51.100.2. So the traffic is denied from 198.51.100.2. You can again modify the ACL configuration to permit traffic from 198.51.100.2.

Once you finished you can shutdown the topology and clean up.

In [10]:
# close our telnet
loginr1.close()

In [11]:
# Close the sim
sim.stop()
sim.clean()

INFO:pyvxr.vxr:Stopping sim on host localhost (dir /nobackup/vxr/pyvxr/68ot1y7amv)
INFO:pyvxr.sim:Stopping previous simulation (if any)
INFO:pyvxr.vxr:Cleaning sim on host localhost (dir /nobackup/vxr/pyvxr/68ot1y7amv)
INFO:pyvxr.sim:Stopping previous simulation (if any)
INFO:pyvxr.sim:Cleaning previous simulation (if any)


> Clean up the sim scratch-space - delete the sim directory 

In [12]:
shutil.rmtree(sim_dir)